In [87]:
import pandas as pd
import numpy as np
import os

In [88]:
path = r'/Users/calebhill/09:01:2023 - Instacart Basket Analysis/02 Data/Prepared Data'

In [89]:
filename = 'orders_products_merged_2.0.pkl'

In [90]:
filepath = os.path.join(path, filename)

In [91]:
ords_prods_merge = pd.read_pickle(filepath)

In [92]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

## 2) In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [93]:
#outputs the average order number for each department
ords_prods_merge.groupby(['department_id']).agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## 3) Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

On my subset I noticed that it differed from the one in the lesson example and I know that depending on how the file was organized, the first 1000000 rows would differ resulting in a different average. The only way to get the whole picture is to use the whole data set.

## 4) Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [94]:
#the next three cells create the flag based on the criteria
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [95]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [96]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

## 5) The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [97]:
#turns the outliers over 100 into NaN
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [98]:
#outputs some basic statistic for the new loyalty_flag column
print(ords_prods_merge.groupby(['loyalty_flag']).agg({'prices': ['mean', 'max', 'min','std']}))

                    prices                     
                      mean   max  min       std
loyalty_flag                                   
Loyal customer    7.773575  25.0  1.0  4.206151
New customer      7.801206  25.0  1.0  4.266834
Regular customer  7.798262  25.0  1.0  4.254886


## 6) The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
-If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

-If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [99]:
#the next three cells create the flag based on the criteria
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [100]:
ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'spending_flag'] = 'Low spender'

In [101]:
ords_prods_merge.loc[ords_prods_merge['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [102]:
#confirms the output is as expected
ords_prods_merge[['user_id', 'average_price', 'spending_flag']].head(100)

,user_id,average_price,spending_flag
0,138,6.935811,Low spender
1,138,6.935811,Low spender
2,709,7.930208,Low spender
3,764,4.972414,Low spender
4,764,4.972414,Low spender
...,...,...,...
95,10831,7.494286,Low spender
96,11119,5.878641,Low spender
97,11186,6.618085,Low spender
98,11243,5.927149,Low spender


In [103]:
#checks frequency of the new column
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     32285131
High spender      119728
Name: count, dtype: int64

## 7) In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
-If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

-If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

-If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [104]:
#creates a median frequency column to use when creating the flag
ords_prods_merge['median_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_last_order'].transform(np.median)

In [105]:
#the next 3 cells create the flag based on the given criteria
ords_prods_merge.loc[ords_prods_merge['median_frequency'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

In [106]:
ords_prods_merge.loc[(ords_prods_merge['median_frequency'] >10) & (ords_prods_merge['median_frequency'] <= 20), 'order_frequency_flag'] = 'Regular Customer'

In [107]:
ords_prods_merge.loc[ords_prods_merge['median_frequency'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [108]:
#confirms results are as expected
ords_prods_merge[['user_id', 'median_frequency', 'order_frequency_flag']].head(2000)

,user_id,median_frequency,order_frequency_flag
0,138,8.0,Frequent customer
1,138,8.0,Frequent customer
2,709,8.0,Frequent customer
3,764,9.0,Frequent customer
4,764,9.0,Frequent customer
...,...,...,...
1995,30781,15.0,Regular Customer
1996,30781,15.0,Regular Customer
1997,30781,15.0,Regular Customer
1998,30781,15.0,Regular Customer


In [109]:
ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        21559853
Regular Customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [111]:
ords_prods_merge[ords_prods_merge['order_frequency_flag'].isnull()][['user_id', 'days_since_last_order', 'median_frequency', 'order_frequency_flag', 'order_number']]

,user_id,days_since_last_order,median_frequency,order_frequency_flag,order_number
6234909,159838,NaN,NaN,NaN,1
12947653,159838,NaN,NaN,NaN,1
13839012,159838,NaN,NaN,NaN,1
14758536,159838,NaN,NaN,NaN,1
21673807,159838,NaN,NaN,NaN,1


These are all first time orders so the missing values are expected

## 9) Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [112]:
ords_prods_merge.to_pickle(os.path.join(path, 'orders_products_merged_3.0.pkl'))